### 신문기자 챗봇 만들기
 - 랭그래프도 펑션 콜링(도구 호출) 가능
 - 사용자가 주제를 제시하면 인터넷을 검색하여 최신 이슈를 기반으로 세부 주제를 선정하고, 그 주제를 검색해서 최종 기사를 작성하는 신문기자 챗봇 만들기

In [1]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.01)
model.invoke('안녕하세요')

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BqbT77k5NWunwjseZceHM8Ko2l7kv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--29dfc7e3-da5b-4fe7-b692-5684d68f439d-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
# 상태 설정
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    """
    State 클래스는 TypedDict를 상속받습니다.

    속성:
        messages (Annotated[list[str], add_messages]): 메시지들은 "list" 타입을 가집니다.
        주석에 있는 'add_messages' 함수는 이 상태 키가 어떻게 업데이트되어야 하는지를 정의합니다.
        (이 경우, 메시지를 덮어쓰는 대신 리스트에 추가합니다.)
    """
    messages: Annotated[list[str], add_messages]

# StateGraph 클래스를 사용하여 State 타입의 그래프 생성
graph_builder = StateGraph(State)

In [6]:
# 도구 등록하기
from langchain_core.tools import tool
from datetime import datetime
import pytz
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

import bs4
from langchain_community.document_loaders import WebBaseLoader

# 도구 함수 정의
@tool
def get_current_time(timezone: str, location: str) -> str:
    """현재 시각을 반환하는 함수."""
    try:
        tz = pytz.timezone(timezone)
        now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        result = f'{timezone} ({location}) 현재 시각 {now}'
        return result
    except pytz.UnknownTimeZoneError:
        return f"알 수 없는 타임존: {timezone}"
    
@tool
def get_web_search(query: str, search_period: str='w') -> str: # 주석은 주석일 뿐 기본값은 정해줘야 한다. 주석하고 내용은 일치시킬 것
    """
    웹 검색을 수행하는 함수.

    Args:
        query (str): 검색어
        search_period (str): 검색 기간 (e.g., "w" for past week (default), "m" for past month, "y" for past year, "d" for pasr day)

    Returns:
        str: 검색 결과
    """
    wrapper = DuckDuckGoSearchAPIWrapper(
        region="kr-kr",
        time=search_period
    )

    print('\n------- WEB SEARCH -------')
    print(query)
    print(search_period)

    search = DuckDuckGoSearchResults(
        api_wrapper=wrapper,
        # source = "news"
        results_separator=';\n'
    )

    searched = search.invoke(query)

    for i, result in enumerate(searched.split(';\n')):
        print(f'{i+1}. {result}')
    
    return searched

# 도구 바인딩
tools = [get_current_time, get_web_search]

In [7]:
# get_current_time 테스트
tools[0].invoke({"timezone": "Asia/Seoul", "location": "서울"})

'Asia/Seoul (서울) 현재 시각 2025-07-07 17:48:44'

In [9]:
# get_web_search 테스트
tools[1].invoke({"query": "파이썬", "search_period": "m"})


------- WEB SEARCH -------
파이썬
m
1. snippet: The official home of the Python Programming Language, title: Welcome to Python.org, link: https://www.python.org/
2. snippet: 1 day ago · 파이썬은 우아한 문법과 동적 타이핑 (typing)을 지원하는 인터프리터 언어로서, 대부분 플랫폼과 다양한 문제 영역에서 스크립트 작성과 빠른 응용 프로그램 개발에 이상적인 환경을 …, title: 파이썬 자습서 — Python 3.13.5 문서, link: https://docs.python.org/ko/3/tutorial/index.html
3. snippet: Jul 24, 2020 · 지금까지 총 29회에 걸친 Python 기초 편 포스팅 마무리로, 파이썬 전체 명령어 모음 및 요약, 데이터 타입 종류 및 기본/컬렉션형 구분 요약, 연산자의 의미 및 사용 예, 함수의 종류 …, title: 파이썬 요약 및 정리 - 명령어, 데이터 타입, 연산자, 함수 및 ..., link: https://m.blog.naver.com/youndok/222040514693
4. snippet: 2 days ago · PyPy: 파이썬으로 작성된 파이썬 인터프리터. 현대의 파이썬은 여전히 인터프리터 언어처럼 동작하나 사용자가 모르는 사이에 스스로 파이썬 소스 코드를 컴파일하여 바이트 …, title: 파이썬 - 위키백과, 우리 모두의 백과사전, link: https://ko.wikipedia.org/wiki/파이썬


'snippet: The official home of the Python Programming Language, title: Welcome to Python.org, link: https://www.python.org/;\nsnippet: 1 day ago · 파이썬은 우아한 문법과 동적 타이핑 (typing)을 지원하는 인터프리터 언어로서, 대부분 플랫폼과 다양한 문제 영역에서 스크립트 작성과 빠른 응용 프로그램 개발에 이상적인 환경을 …, title: 파이썬 자습서 — Python 3.13.5 문서, link: https://docs.python.org/ko/3/tutorial/index.html;\nsnippet: Jul 24, 2020 · 지금까지 총 29회에 걸친 Python 기초 편 포스팅 마무리로, 파이썬 전체 명령어 모음 및 요약, 데이터 타입 종류 및 기본/컬렉션형 구분 요약, 연산자의 의미 및 사용 예, 함수의 종류 …, title: 파이썬 요약 및 정리 - 명령어, 데이터 타입, 연산자, 함수 및 ..., link: https://m.blog.naver.com/youndok/222040514693;\nsnippet: 2 days ago · PyPy: 파이썬으로 작성된 파이썬 인터프리터. 현대의 파이썬은 여전히 인터프리터 언어처럼 동작하나 사용자가 모르는 사이에 스스로 파이썬 소스 코드를 컴파일하여 바이트 …, title: 파이썬 - 위키백과, 우리 모두의 백과사전, link: https://ko.wikipedia.org/wiki/파이썬'

In [10]:
# 도구 목록 확인
for tool in tools:
    print(tool.name, tool)

get_current_time name='get_current_time' description='현재 시각을 반환하는 함수.' args_schema=<class 'langchain_core.utils.pydantic.get_current_time'> func=<function get_current_time at 0x000001E48965D080>
get_web_search name='get_web_search' description='웹 검색을 수행하는 함수.\n\nArgs:\n    query (str): 검색어\n    search_period (str): 검색 기간 (e.g., "w" for past week (default), "m" for past month, "y" for past year, "d" for pasr day)\n\nReturns:\n    str: 검색 결과' args_schema=<class 'langchain_core.utils.pydantic.get_web_search'> func=<function get_web_search at 0x000001E48965D300>


In [ ]:
# 모델에 도구 연결 및 generate 노드 생성 
model_with_tools = model.bind_tools(tools)

def generate(state: State):
    """
    주어진 상태를 기반으로 챗봇의 응답 메시지를 생성합니다.

    매개변수:
    state (State): 현재 대화 상태를 나타내는 객체로, 이전 메시지들이 포함되어 있습니다.

    반환값:
    dict: 모델이 생성한 응답 메시지를 포함하는 딕셔너리
        형식은 {"messages": [응답 메시지]}입니다.
    """
    return {"messages": model_with_tools.invoke(state["messages"])}

graph_builder.add_node("generate", generate)